In [1]:
# install required packages

import numpy as np 
import pandas as pd 
import os
import seaborn as sns
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from scipy import stats
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

In [2]:
# read database and store in a dataframe variable
def read(path='/kaggle/input/10000-amazon-products-dataset/Amazon_Products.csv'):
    df = pd.read_csv(path, dtype=str)
    display(df.head())
    return df

df = read()
df.shape

## Data Cleaning

There are a wide variety of columns available in the dataset which will need to be looked at one at a time. Upon a quick look, it seems like most of the columns starting with "Unnamed" are not useful and can be deleted.

In [3]:
# function to summarize any column
def ColSummary (col, dtype=None, new_col=None):
    try:
        print("\n"+str(col).upper()+" ANALYSIS")
        if new_col:
            df.rename(columns={col:new_col})
            print("\t"+col+" name updated to "+new_col)
            col = new_col
        if dtype:
            df[col] = df[col].astype(dtype)
            print("\tType changed to "+dtype)
        print('\tTotal number of values : ', len(df[col]))
        print('\tTotal number of unique values : ', df[col].nunique())
        print('\tUnique values : ',df[col].unique()[:5])
        print('\tRange of length : ', min(df[col].apply(lambda x: len(str(x))))," to ",max(df[col].apply(lambda x: len(str(x)))))
        print('\tMissing Values : ', df[col].isna().sum())
    except:
        print("\n*** Cannot work with this datatype.. Try Again.... ***")

In [4]:
# delete columns which are unnamed
unnamed_cols = [col for col in df.columns if 'Unnamed' in col]
df = df.drop(unnamed_cols, axis = 1)
print("Leftover columns: ", df.columns)

In [5]:
# Analyze and clean Unique IDS (primary key)
ColSummary('uniq_id', 'str')
# Some IDs have unusual length
print('\n\tInspecting IDs with unusual length : ', list(df['uniq_id'][df['uniq_id'].apply(lambda x: len(str(x)))<=3]))
print('\tDeleting IDs with unusual length ..... ')
df = df.drop(index = list(df['uniq_id'][df['uniq_id'].apply(lambda x: len(str(x))<=3)].index))
print('\nUnique IDs updated .....')
ColSummary('uniq_id')

In [6]:
# Analyze Product Name - No cleaning required
ColSummary('product_name', 'string')

In [7]:
# Analyze Manufacturer - No cleaning required
ColSummary('manufacturer', 'string') 

In [8]:
# Analyze Price - Need to convert string to numeric
ColSummary('price')

# Some rows have range mentioned instead of a price - this is significantly changing the df size
# another way to tackle this would be to add the middle point of the range as the prize
print("\n Deleting rows which have price with greater than 5 length i.e. have ranges instead of values .....")
df = df.drop(index = list(df['price'][df['price'].apply(lambda x: len(str(x))>5)].index))

# Remove the euro sign before numeric conversion
df['price'] = df['price'][df['price'].isna()==False].str[1:]

# Convert into float and analyze updated column
ColSummary('price', 'float64')

In [9]:
# Analyze Number available in stock - need to change to numeric 
ColSummary('number_available_in_stock')

# split the stings into list and keep only the numeric part
df['number_available_in_stock'] = df['number_available_in_stock'].apply(lambda x: str(x).split()[0])

# change to float and analyze again
ColSummary('number_available_in_stock','float64')

In [10]:
# Analyze Number of Reviews - need to convert into numeric
ColSummary('number_of_reviews')

# some strings have commas which is preventing direct float conversion
print("\n Removing commas from strings for float conversion .....")
df['number_of_reviews'] = df['number_of_reviews'].apply(lambda x: str(x).replace(',',''))

# convert to float and analyze updated columns
ColSummary('number_of_reviews', 'float64')

In [11]:
# Analyze Number of Answered Questions - need to convert into numeric
ColSummary('number_of_answered_questions','float64')

In [12]:
# Analyze Avg Review Ratign - need to convert into numeric
ColSummary('average_review_rating')

# the values include the denominator of the rating which is consistent throughtout and hence, need not be mentioned
df['average_review_rating'] = df['average_review_rating'].apply(lambda x: str(x).split()[0])

# After retaining only ratings and removing additional text, convert into float
ColSummary('average_review_rating', 'float64')

In [13]:
# Analyze Categories - Each value has all the subcatogories mentioned in a string format - need to convert to list
ColSummary('amazon_category_and_sub_category')

# break using delimiter for list conversion
df['amazon_category_and_sub_category'] = df['amazon_category_and_sub_category'].apply(lambda x: str(x).split(" > "))

# each list has varied length, for easier analysis - make len consistent by adding None
itr = max(df['amazon_category_and_sub_category'].apply(lambda x: len(x)))
for i in range (1, itr+1):
    df['amazon_category_and_sub_category'].apply(lambda x: x.append("None") if (len(x) < i) else x)

# Analyze updated column
ColSummary('amazon_category_and_sub_category')

In [14]:
# Analyze description - no change required
ColSummary('description')

In [15]:
# Analyze Product Information - no change required
ColSummary('product_information')

In [16]:
# Analyze Product Description - no change required

# the values look similar to description column - this column might not add much value 
ColSummary('product_description')

In [17]:
# Analyze URLs of the next bought product 

ColSummary('items_customers_buy_after_viewing_this_item')

In [18]:
# Analyze customer Qs 
ColSummary('customer_questions_and_answers')

# There are a lot of missing values in this column
# This is not unusual as customers might not have questions about each product

In [19]:
# Final df shape after cleaning
df.shape

## Exploratory Data Analysis (EDA)
After cleaning the data, we can run some exploratory analysis to understand the data better. I am going to simply follow my curiousity here.

<font color = 'blue'>
<h2> Question 1. What are the most popular brands? </h2>

There are multiple ways to define "popular". I'm going to explore 3 ways here:
<ul>
    <li>Number of products sold</li>
<li>Total sales by price </li>
<li>Highest rated products </li>
    </ul>

In [20]:
# number of products sold

df[['product_name','manufacturer']].groupby('manufacturer').count().sort_values(by = "product_name", ascending = False)

In [21]:
# Total sales by price

df[['price','manufacturer']].groupby('manufacturer').sum().sort_values(by = "price", ascending = False)

In [22]:
# highest rated

ratings_df = df[['average_review_rating', 'number_of_reviews','manufacturer']].groupby('manufacturer').agg({'average_review_rating':'mean', 'number_of_reviews': "sum"
                                                                                      }).sort_values(by = ["average_review_rating","number_of_reviews"], ascending = False)
ratings_df

In [23]:
# checking number of manufacturers with 5 rating
print("\n Brands with perfect 5 rating:", len(ratings_df[ratings_df["average_review_rating"]==5.0]))

# checking number of 5-rated manufactureres which have > 5 number of reviews      
print("\n Brands with perfect 5 rating and substantial number of reviews:",
      len(ratings_df[ratings_df["average_review_rating"]==5.0][ratings_df[ratings_df["average_review_rating"]==5.0]
                                                               ["number_of_reviews"]>5]),"\n\n")

<font color = 'blue'>
    <h3> Results </h3>

- Oxford Diecast, Disney and MyTinyWorld have sold the most products
- Oxfort Diecast has had the highest sales by far almost double the second highest
- 669 manufacturers have perfect 5 rating but only 22 have been rated 5 on average by 5 or more customers

***


<font color = 'blue'>
<h2> Question 2. What are some keywords that are repeatedly used by brands in descriptions? </h2>

In [24]:
def show_wordcloud(data, title = None):
    stopwords = set(STOPWORDS)
    mask = np.array(Image.open("../input/amazon/amazon.png"))
    wordcloud = WordCloud(
        background_color='white',
        stopwords = stopwords,
        mask = mask,
        max_words=200,
        max_font_size=40, 
        scale=1,
        random_state=1
).generate(" ".join(data))

    fig = plt.figure(1, figsize=(25, 15))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

In [25]:
col = 'description'
wordcloud_df = df[df[col].notna()]
show_wordcloud(wordcloud_df[col].values)

<font color = 'blue'>
    <h3> Results </h3>

Some interesting words repeated across descriptions are: Suitable, high quality, classic, great, feature, card

***

<font color = 'blue'>
<h2> Question 3. Predict the amazon categories using description? </h2>

In [26]:
def TextClassifier(text_data,label):
    "function to create text classifier for a set of data and labels"
    text_clf = Pipeline([ ('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),('clf', MultinomialNB())])
    text_clf = text_clf.fit(text_data, label)
    return text_clf

In [27]:
# defining the target variable

df['category'] = df['amazon_category_and_sub_category'].apply(lambda x: x[0])

categorytolabel = {}
i = 1
for item in df['category'].unique():
    categorytolabel[item] = i
    i = i+1

df['category_labels'] = df['category'].map(categorytolabel)

df = df.groupby('category').filter(lambda x : len(x)>30)
df = df.drop(index = list(df[df['category']=='nan'].index))

In [28]:
# running the model and comparing the predictions
text_df = df[['product_information','product_description']]

for col in text_df.columns:
    text_df[col] = text_df[col].fillna(value='None')
    model = TextClassifier(text_df[col],df['category_labels'])
    text_df[col+'_classification'] = model.predict(text_df[col])

text_df['product_information_classification'] = text_df['product_information_classification'].map({value : key for (key, value) in categorytolabel.items()})
text_df['product_description_classification'] = text_df['product_description_classification'].map({value : key for (key, value) in categorytolabel.items()})
df = df.merge(text_df[['product_information_classification','product_description_classification']], how = 'left', left_index = True, right_index = True)


from sklearn.metrics import classification_report
print("\n\n ********* CLASSIFICATION ACCURACY BASED ON PRODUCT INFORMATION *********")
print("\n", classification_report(df['category'], df['product_information_classification'] ))
print("\n\n ********* CLASSIFICATION ACCURACY BASED ON PRODUCT DESCRIPTION *********")
print("\n", classification_report(df['category'], df['product_description_classification'] ))

<font color = "blue">
<h3>Results</h3>
Product Description is a better predictor of amazon category compared to product information

***